In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
from PIL import Image
import torch
import numpy as np
from torch import nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
import torchvision.models as models
from torch.utils.data import random_split
from sklearn.neighbors import NearestNeighbors

In [ ]:
import cv2
from PIL import Image
import torch
import numpy as np
from torch import nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
import torchvision.models as models
from torch.utils.data import random_split
from sklearn.neighbors import NearestNeighborsdevice = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device available now:', device)
df_train = pd.read_csv("../input/shopee-product-matching/train.csv")

In [ ]:
df_train = pd.read_csv("../input/shopee-product-matching/train.csv")
df_train.head(3)

In [ ]:
#Augmentation data x8
for i in range(3):
    df_train = pd.concat([df_train, df_train], ignore_index=True)
    df_train.reset_index(drop=True)

In [ ]:
tmp = df_train.groupby('label_group').posting_id.agg('unique').to_dict()
df_train['target'] = df_train.label_group.map(tmp)
df_train.head(3)

## Creating shopee dataset

In [ ]:
class ShopeeDataset(Dataset):
    
    def __init__(self, csv, train=True):
        self.csv = csv.reset_index()
        self.train = train
        self.transform_random = transforms.Compose([
                                            transforms.RandomHorizontalFlip(),
                                            transforms.RandomVerticalFlip(),
                                            transforms.RandomRotation(90),
                                            transforms.Resize((256, 256)),
                                            transforms.ToTensor(),])
        
    def __len__(self):
        return len(self.csv)
    
    def __getitem__(self, index):
        if self.train:
            image = Image.open('../input/shopee-product-matching/train_images/' +
                               self.csv.image[index])
        else:
            image = Image.open('../input/shopee-product-matching/test_images/' +
                               self.csv.image[index])
        

        image = self.transform_random(image)
        image = torch.div(image, 255)
        
        if self.train:
            label = torch.tensor(self.csv.label_group[index])
            return image, label
        
        else:
            return image

In [ ]:
train_dataset = ShopeeDataset(csv=df_train)
train_dataloader = DataLoader(train_dataset, batch_size=16, num_workers=4)

print("Dataset length: {}".format(len(train_dataset)), "\n" +
      "Shape of images: {}".format(train_dataset[0][0].shape))

## Importing model ResNet18

In [ ]:
resnet18 = models.resnet18(pretrained=True)

## Extracting embeddings from train images using ResNet18

In [ ]:
def extract_embeddings(model, dataloader):
    embeddings = []

    with torch.no_grad():
        for image, label in tqdm(dataloader):
            if torch.cuda.is_available():
                image = image.to('cuda')
                model.to('cuda')

            img_emd = model(image)
            img_emd = img_emd.detach().cpu().numpy()
            embeddings.append(img_emd)

    embeddings = np.concatenate(embeddings)
    print("Shape of embeddings: {}".format(embeddings.shape))
    
    return embeddings

In [ ]:
#embeddings = extract_embeddings(resnet18, train_dataloader)

In [ ]:
np.save("image_embeddings", embeddings)

## Making predictions

In [ ]:
def get_f1_score(y, y_pred):
    intr_sect = np.array([len(set(x1).intersection(x2)) for x1, x2 in zip(y, y_pred)])
    len_y = y.apply(lambda x: len(x)).values
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    f1 = 2 * intr_sect / (len_y + len_y_pred)
    
    return f1

In [ ]:
def predict(model, val_set, train_emd, val_emd, train_df, val_df, threshold):
    predictions = []
    
    knn_model = NearestNeighbors(n_neighbors=50, algorithm='brute', metric='cosine')
    knn_model.fit(train_emd)
    dists, idx = knn_model.kneighbors(val_emd)
    
    for i in range(val_emd.shape[0]):
        mask = dists[i] < threshold
        pred = train_df.loc[idx[i][mask]].posting_id.values
        predictions.append(pred)
        
    val_df['prediction'] = predictions
    val_df['f1_score'] = get_f1_score(val_df['target'], val_df['prediction'])
    score = val_df['f1_score'].mean()
    
    return score

In [ ]:
num_imgs = len(train_dataset)
num_train_imgs = int(num_imgs*.8)
idx = np.array(range(num_imgs))
np.random.shuffle(idx)
idx_train, idx_val = idx[:num_train_imgs], idx[num_train_imgs:]

In [ ]:
train_set = ShopeeDataset(csv=df_train.loc[idx_train])
train_loader = DataLoader(train_set, batch_size=16, num_workers=4)

val_set = ShopeeDataset(csv=df_train.loc[idx_val])
val_loader = DataLoader(val_set, batch_size=16, num_workers=4)

In [ ]:
train_embd = extract_embeddings(resnet18, train_loader)
val_embd = extract_embeddings(resnet18, val_loader)

In [ ]:
train_df = df_train.loc[idx_train].reset_index(drop=True)
val_df = df_train.loc[idx_val].reset_index(drop=True)

In [ ]:
threshold = .3
f1_score = predict(resnet18, val_set, train_embd, val_embd, train_df, val_df, threshold)
print('F1 score: {}'.format(f1_score))

# EXPLORE DATA

In [ ]:
import cv2
from PIL import Image
import torch
import numpy as np
from torch import nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
import torchvision.models as models
from torch.utils.data import random_split
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import matplotlib.pyplot as plt

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import pos_tag, ne_chunk
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm
from textblob import TextBlob


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device available now:', device)

In [ ]:
df_train = pd.read_csv("../input/shopee-product-matching/train.csv")
df_test = pd.read_csv("../input/shopee-product-matching/test.csv")
print(len(df_train))
print(len(df_test))

In [ ]:
df_train.head(3)

In [ ]:
#See different titles for same product

df = pd.DataFrame(index = [0, 1, 2], columns = ["Title 1", "Title 2"])
labels = [3648931069, 1565741687, 249114794]
n= len(labels)
for l in range(n):
    indexes = df_train[df_train["label_group"]==labels[l]].index.values 
    titles = df_train["title"].iloc[indexes].values
    df.iloc[l] = titles

df

In [ ]:
fig, ax = plt.subplots(1,2)
img_A = np.array(Image.open('../input/shopee-product-matching/train_images/' +  df_train["image"].iloc[indexes[0]]))
img_B = np.array(Image.open('../input/shopee-product-matching/train_images/' +  df_train["image"].iloc[indexes[1]]))
ax[0].imshow(img_A)
ax[1].imshow(img_B)

# PREPROCESS

In [ ]:
res = 256

transform = transforms.Compose([transforms.Resize((res, res)),
                                transforms.ToTensor(),])
image = Image.open('../input/shopee-product-matching/train_images/' +  df_train["image"].iloc[indexes[1]])
image_ = transform(image)

fig, ax = plt.subplots(1,2)
ax[0].imshow(image)
ax[1].imshow(image_.permute(1,2,0))

In [ ]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
    
# Preprocess titles
original_titles = df_train["title"]
lemmatizer = WordNetLemmatizer()

def preprocess_title(title):
    title_clean = title.lower().translate(str.maketrans('','',string.punctuation)).strip()
    title_tokenize = word_tokenize(title_clean)
    title_w_stopword = [w for w in title_tokenize if not w in stopwords.words()]
    title_lemmatized = [lemmatizer.lemmatize(w) for w in title_w_stopword]
    return " ".join(title_lemmatized)

def pos_title(title):
    return TextBlob(' '.join(title)).tags
    
for i in [0, 1, 2, 4, 5]:
    print(color.BOLD + "Orginial Title -" + color.END, original_titles.iloc[i])
    print(color.BOLD + "Preprocessed Title -" + color.END, preprocess_title(original_titles.iloc[i]))
    print('\n')
#print("Part Of Speech : ", pos_title(preprocess_title(original_titles.iloc[5])))


In [ ]:
tqdm.pandas()

df_train["preprocess_title"] = df_train["title"].progress_apply(lambda x: preprocess_title(x))
df_train["preprocess_title"] = df_train["preprocess_title"].astype('U')

In [ ]:
# Get title embeddings
df_train["preprocess_title"] = df_train["preprocess_title"].astype('U')
preprocess_title = df_train["preprocess_title"].values
tf_idf = TfidfVectorizer(stop_words='english', binary=True, max_features=25000)
text_embeddings = tf_idf.fit_transform(preprocess_title).toarray()

In [ ]:
import cupy 


predictions = []
BATCH = 4096
N = len(text_embeddings) 
print(N)

print("Title based prediction...")

for i in range(N//BATCH):
    
    a = i * BATCH
    b = min((i+1) * BATCH, N)
    print("BATCH : ", a, "-", b)
    
    # Cosine similarity distance
    cts = cupy.matmul(text_embeddings, text_embeddings[a:b].T).T
    
    for k in range(b-a):
        index = cupy.where(cts[k,] > 0.7)[0]
        index = cupy.asnumpy(index)
        pred = df_train.iloc[index]["posting_id"].values
        
        predictions.append(pred)

print(predictions)